In [20]:
# load packages
import os
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import pcntoolkit as pcn
import pickle

from scipy.stats import shapiro

from enigmatoolbox.utils.parcellation import parcel_to_surface
from enigmatoolbox.plotting import plot_cortical, plot_subcortical
from enigmatoolbox.utils.useful import reorder_sctx

from pcntoolkit.normative import estimate, predict, evaluate
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix

ext_scripts_dir = ('/home/barbora/Documents/Projects/Normative_Models/ESO/braincharts/scripts')
os.chdir(ext_scripts_dir)

from nm_utils import remove_bad_subjects, load_2d

code_dir = ('/home/barbora/Documents/Projects/Normative_Models/ESO/code')
os.chdir(code_dir)

# importing custom functions
from clinics_desc_functions import prepare_data, plot_quality, trajectory_plotting, dk_roi_viz, load_clinics, en_qc, pretrained_adapt_small, set_seed

# set seed
set_seed()

# formatiing
%matplotlib inline
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}



Random seed 42 has been set.


In [129]:
# where things are
main_dir = ('/home/barbora/Documents/Projects/Normative_Models/ESO')
models_dir = ('/home/barbora/Documents/Projects/Normative_Models/ESO/models/control_stability')
os.makedirs(models_dir, exist_ok=True)
cdata_dir = ('/home/barbora/Documents/Projects/2021_06_AZV_ESO/data')
fsdata_dir = ('/home/barbora/Documents/Projects/Normative_Models/ESO/fs_stats')
bdata_dir = ('/home/barbora/Documents/Projects/Normative_Models/ESO/backup')
images_dir = ('/home/barbora/Documents/Projects/Normative_Models/ESO/models/control_stability/img')
os.makedirs(images_dir, exist_ok=True)
pretrained_dir = ('/home/barbora/Documents/Projects/Normative_Models/ESO/braincharts')

In [22]:
####
# Getting a pretrained model
# ###
model_name = 'lifespan_57K_82sites'
site_names = 'site_ids_82sites.txt'

# load a set of site ids from this model. This must match the training data
with open(os.path.join(pretrained_dir,'docs', site_names)) as f:
    site_ids_tr = f.read().splitlines()

## Our dataset currently misses 2 variables: 'lh_MeanThickness_thickness', 'rh_MeanThickness_thickness'; to be extracted from a2009s
# load the list of idps for left and right hemispheres, plus subcortical regions
with open(os.path.join(pretrained_dir,'docs','phenotypes_lh.txt')) as f:
    idp_ids_lh = f.read().splitlines()
with open(os.path.join(pretrained_dir,'docs','phenotypes_rh.txt')) as f:
    idp_ids_rh = f.read().splitlines()
with open(os.path.join(pretrained_dir,'docs','phenotypes_sc.txt')) as f:
    idp_ids_sc = f.read().splitlines()

# we choose here to process all idps
idp_ids = idp_ids_lh + idp_ids_rh + idp_ids_sc

# delete features that are not present in our data
idp_ids.remove('lh_MeanThickness_thickness')
idp_ids.remove('rh_MeanThickness_thickness')


In [23]:
# load clinics from the first visit
c_v1 = pd.read_excel(os.path.join(cdata_dir,'visit1_desc.xlsx')) 
c_v1 = load_clinics(c_v1)
d_v1 = pd.read_csv(os.path.join(bdata_dir,'fit_external_thickness_1.txt'), delimiter=';', index_col=0)
v1_concat = pd.concat([c_v1, d_v1], axis=1, join="inner")
v1_concat["sitenum"] =1000

# load clinics from the second visit
c_v2 = pd.read_excel(os.path.join(cdata_dir,'visit2_desc.xlsx')) 
c_v2 = load_clinics(c_v2)
d_v2 = pd.read_csv(os.path.join(bdata_dir,'fit_external_thickness_2.txt'), delimiter=';',  index_col=0)
v2_concat = pd.concat([c_v2, d_v2], axis=1, join="inner")
v2_concat["sitenum"] =1000

In [24]:
# Quality control based on Euler Number
save_img, img_dir, show_img = True, images_dir, False
v1_clean = en_qc(v1_concat, save_img=save_img, img_dir=img_dir, show_img=show_img)
v2_clean = en_qc(v2_concat, save_img=save_img, img_dir=img_dir, show_img=show_img)

# remove visit from index
v1_clean.index = v1_clean.index.str.slice_replace(start=-2, repl='').to_numpy()
v2_clean.index = v2_clean.index.str.slice_replace(start=-2, repl='').to_numpy()

# sort 
v1_clean = v1_clean.sort_index(ascending=True)
v2_clean = v2_clean.sort_index(ascending=True)

In [25]:
# Splitting datasets
v1_pat = v1_clean[v1_clean['Category']=='Patient']
v1_cont = v1_clean[v1_clean['Category']=='Control']
#v1_cont_train = v1_cont.sample(frac = 0.5)
#v1_cont_ad = v1_cont.drop(v1_cont_train.index)

# extract a list of unique site ids from the test set
site_ids_te =  sorted(set(v1_pat['site'].to_list()))

v2_pat = v2_clean[v2_clean['Category']=='Patient']
v2_cont = v2_clean[v2_clean['Category']=='Control']
#v2_cont_train = v2_cont.sample(frac = 0.5)
#v2_cont_ad = v2_cont.drop(v2_cont_train.index)

In [26]:
# find intersection of v1 and v2 - we are going to loop over these people
common = v1_cont.index.intersection(v2_cont.index)

v1_common_f = v1_cont.index[np.where(np.bitwise_and(v1_cont['sex'] == 0, v1_cont.index.isin(common)))[0]]
v2_common_f = v2_cont.index[np.where(np.bitwise_and(v2_cont['sex'] == 0, v2_cont.index.isin(common)))[0]]

v1_common_f_id = np.where(np.bitwise_and(v1_cont['sex'] == 0, v1_cont.index.isin(common)))[0]
v2_common_f_id = np.where(np.bitwise_and(v2_cont['sex'] == 0, v2_cont.index.isin(common)))[0]

v1_common_m = v1_cont.index[np.where(np.bitwise_and(v1_cont['sex'] == 1, v1_cont.index.isin(common)))[0]]
v2_common_m = v2_cont.index[np.where(np.bitwise_and(v2_cont['sex'] == 1, v2_cont.index.isin(common)))[0]]

v1_common_m_id = np.where(np.bitwise_and(v1_cont['sex'] == 1, v1_cont.index.isin(common)))[0]
v2_common_m_id = np.where(np.bitwise_and(v2_cont['sex'] == 1, v2_cont.index.isin(common)))[0]


In [14]:
###
# Running models
# leave one out crossvalidation for all subjects that have both visits
###

qm = ['EXPV_predict.txt', 'MSLL_predict.txt', 'pRho_predict.txt', 'Rho_predict.txt',  'RMSE_predict.txt',  'SMSE_predict.txt']
qm_colnames = ['EXPV', 'MSLL', 'pRho', 'Rho', 'RMSE', 'SMSE']

# initialize empty arrays
for idp in idp_ids: #['lh_G&S_paracentral_thickness']: 
    #idp = 'lh_G&S_paracentral_thickness'
    print(idp)

    # empty arrays for idp
    v1_yhat, v2_yhat, v1_ys2, v2_ys2, v1_Z,v2_Z  = np.empty(len(common)), np.empty(len(common)), np.empty(len(common)), np.empty(len(common)), np.empty(len(common)), np.empty(len(common))
    quality = np.empty([len(common),6])

    # create folder structure
    idp_dir = os.path.join(pretrained_dir,'models','lifespan_57K_82sites', idp)
    idp_cv_dir = os.path.join(models_dir,idp)
    os.makedirs(idp_cv_dir, exist_ok=True)
    os.chdir(idp_cv_dir)


    for icont in range(0, len(common)): # we are only running the cross-validation for subjects with both visits
        #icont = 0

        df_ad = v1_cont.drop(index=common[icont])
        df_te = pd.concat([v1_cont[v1_cont.index == common[icont]],v2_cont[v2_cont.index == common[icont]]])

        pretrained_adapt_small(idp, site_ids_tr, site_ids_te, pretrained_dir, idp_cv_dir, idp_dir, df_ad, df_te)

        # add data to arrays
        v1_yhat[icont], v2_yhat[icont] = np.genfromtxt(os.path.join(idp_cv_dir,'yhat_predict.txt'), delimiter=' ')
        v1_ys2[icont], v2_ys2[icont] = np.genfromtxt(os.path.join(idp_cv_dir,'ys2_predict.txt'), delimiter=' ')
        v1_Z[icont], v2_Z[icont] = np.genfromtxt(os.path.join(idp_cv_dir,'Z_predict.txt'), delimiter=' ')
        quality[icont] = [np.genfromtxt(i).reshape(1)[0] for i in qm]
    
    # write
    np.savetxt(os.path.join(idp_cv_dir, 'v1_yhat.txt'), v1_yhat)    
    np.savetxt(os.path.join(idp_cv_dir, 'v2_yhat.txt'), v2_yhat)    
    np.savetxt(os.path.join(idp_cv_dir, 'v1_ys2.txt'), v1_ys2)    
    np.savetxt(os.path.join(idp_cv_dir, 'v2_ys2.txt'), v2_ys2)    
    np.savetxt(os.path.join(idp_cv_dir, 'v1_Z.txt'), v1_Z)    
    np.savetxt(os.path.join(idp_cv_dir, 'v2_Z.txt'), v2_Z)    
    
    df_quality = pd.DataFrame(quality, columns=qm_colnames)
    df_quality.to_csv(os.path.join(idp_cv_dir, 'quality.txt'), sep=' ', header=True, index=False)   

    # we are only running this to have big design matrices ready for plotting
    pretrained_adapt_small(idp, site_ids_tr, site_ids_te, pretrained_dir, idp_cv_dir, idp_dir, v1_cont, v1_cont[v1_cont.index.isin(common)]) 
    os.rename('cov_bspline_te.txt', 'v1_cov_bspline_te.txt')
    os.rename('resp_te.txt', 'v1_resp_te.txt')
    
    pretrained_adapt_small(idp, site_ids_tr, site_ids_te, pretrained_dir, idp_cv_dir, idp_dir, v1_cont, v2_cont[v2_cont.index.isin(common)]) 
    os.rename('cov_bspline_te.txt', 'v2_cov_bspline_te.txt')
    os.rename('resp_te.txt', 'v2_resp_te.txt')


lh_G&S_frontomargin_thickness
Some sites missing from the training data. Adapting model
Loading data ...
Prediction by model  1 of 1
Evaluating the model ...
Evaluations Writing outputs ...
Writing outputs ...
Some sites missing from the training data. Adapting model
Loading data ...
Prediction by model  1 of 1
Evaluating the model ...
Evaluations Writing outputs ...
Writing outputs ...
Some sites missing from the training data. Adapting model
Loading data ...
Prediction by model  1 of 1
Evaluating the model ...
Evaluations Writing outputs ...
Writing outputs ...
Some sites missing from the training data. Adapting model
Loading data ...
Prediction by model  1 of 1
Evaluating the model ...
Evaluations Writing outputs ...
Writing outputs ...
Some sites missing from the training data. Adapting model
Loading data ...
Prediction by model  1 of 1
Evaluating the model ...
Evaluations Writing outputs ...
Writing outputs ...
Some sites missing from the training data. Adapting model
Loading data

In [183]:
###
# Create dummy data
###

# which sex do we want to plot? 
sex = 1 # 1 = male 0 = female
if sex == 1: 
    clr = 'blue'
else:
    clr = 'red'

# limits for cubic B-spline basis 
xmin = -5 
xmax = 110

# create dummy data for visualisation
print('configuring dummy data ...')
xx = np.arange(xmin, xmax, 0.5)
X0_dummy = np.zeros((len(xx), 2))
X0_dummy[:,0] = xx # intercept
X0_dummy[:,1] = sex # sex covariate

# create the design matrix
X_dummy = create_design_matrix(X0_dummy, xmin=xmin, xmax=xmax, site_ids=None, all_sites=site_ids_tr)

# save the dummy covariates
cov_file_dummy = os.path.join(models_dir,'cov_bspline_dummy_mean.txt')
np.savetxt(cov_file_dummy, X_dummy)

configuring dummy data ...


In [184]:
###
# Plotting normative models
###

sns.set(style='whitegrid')

for idp_num, idp in enumerate(idp_ids): #enumerate(['lh_G&S_paracentral_thickness']):
    print('Running IDP', idp_num, idp, ':')
    idp_dir = os.path.join(pretrained_dir,'models','lifespan_57K_82sites', idp)
    idp_cv_dir = os.path.join(models_dir,idp)
    os.chdir(idp_cv_dir) 

    # load the true data points V1
    v1_yhat_te = load_2d(os.path.join(idp_cv_dir, 'v1_yhat.txt'))
    v1_s2_te = load_2d(os.path.join(idp_cv_dir, 'v1_ys2.txt'))
    v1_y_te = load_2d(os.path.join(idp_cv_dir, 'v1_resp_te.txt'))
    v1_bspline = pd.DataFrame(load_2d(os.path.join(idp_cv_dir, 'v1_cov_bspline_te.txt')))

    # load the true data points V2
    v2_yhat_te = load_2d(os.path.join(idp_cv_dir, 'v2_yhat.txt'))
    v2_s2_te = load_2d(os.path.join(idp_cv_dir, 'v2_ys2.txt'))
    v2_y_te = load_2d(os.path.join(idp_cv_dir, 'v2_resp_te.txt'))
    v2_bspline = pd.DataFrame(load_2d(os.path.join(idp_cv_dir, 'v2_cov_bspline_te.txt')))

    #########################################################################
    # set up the covariates for the dummy data
    print('Making predictions with dummy covariates (for visualisation)')
    yhat, s2 = predict(cov_file_dummy, 
                       alg = 'blr', 
                       respfile = None, 
                       model_path = os.path.join(idp_dir,'Models'), 
                       outputsuffix = '_dummy')
    
    # load the normative model
    with open(os.path.join(idp_dir,'Models', 'NM_0_0_estimate.pkl'), 'rb') as handle:
        nm = pickle.load(handle) 
    
    # get the warp and warp parameters
    W = nm.blr.warp
    warp_param = nm.blr.hyp[1:nm.blr.warp.get_n_params()+1] 

    # warp dummy predictions to create the plots
    med, pr_int = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param)
    
    # extract the different variance components to visualise
    beta, junk1, junk2 = nm.blr._parse_hyps(nm.blr.hyp, X_dummy)
    s2n = 1/beta # variation (aleatoric uncertainty)
    s2s = s2-s2n # modelling uncertainty (epistemic uncertainty)
    #########################################################################


    #  warp predictions for the true data and compute evaluation metrics
    v1_med_te = W.warp_predictions(np.squeeze(v1_yhat_te), np.squeeze(v1_s2_te), warp_param)[0]
    v1_med_te = v1_med_te[:, np.newaxis]

    v2_med_te = W.warp_predictions(np.squeeze(v2_yhat_te), np.squeeze(v2_s2_te), warp_param)[0]
    v2_med_te = v2_med_te[:, np.newaxis]
    #print('metrics:', evaluate(y_te, med_te))
    
    
    ###
    # Adjusting         
    # adjust the data based on the adaptation dataset 
    ###
    #  
    # load the adaptation data
    y_ad = load_2d(os.path.join(idp_cv_dir, 'resp_ad.txt'))
    X_ad = load_2d(os.path.join(idp_cv_dir, 'cov_bspline_ad.txt'))

        # this needs its own crossvalidation cycle
    if sex == 0:
        v1_common_s = v1_common_f
        v2_common_s = v2_common_f
        v1_common_s_id = v1_common_f_id
        v2_common_s_id = v2_common_f_id
    elif sex == 1:
        v1_common_s = v1_common_m
        v2_common_s = v2_common_m
        v1_common_s_id = v1_common_m_id
        v2_common_s_id = v2_common_m_id

    v1_y_rescaled = np.empty([len(v1_common_s),1])
    v2_y_rescaled = np.empty([len(v1_common_s),1])

### Loop through subjects
    for icont in range(0, len(v1_common_s)): # we are only running the cross-validation for subjects with both visits
        
        # pick indicies for adaptation
        idx_a = np.where(np.bitwise_and(v1_cont['sex'] == sex, ~(v1_cont.index == common[icont])))[0] 

        v1_idx = np.where(np.bitwise_and(v1_cont['sex'] == sex, v1_cont.index == common[icont]))[0]    
        v2_idx = np.where(np.bitwise_and(v2_cont['sex'] == sex, v2_cont.index == common[icont]))[0]    

        y_te = np.array([v1_y_te[np.where(v1_bspline[2]==sex)[0][icont]][0],v2_y_te[np.where(v2_bspline[2]==sex)[0][icont]][0]])

        # adjust and rescale the data
        y_te_rescaled, s2_rescaled = nm.blr.predict_and_adjust(nm.blr.hyp, 
                                                                X_ad[idx_a,:], 
                                                                np.squeeze(y_ad[idx_a]), 
                                                                Xs=None, 
                                                                ys=y_te)
        v1_y_rescaled[icont] = y_te_rescaled[0]
        v2_y_rescaled[icont] = y_te_rescaled[1]
    

    # save transformed predictions so that we don't have to sompute this ever again
    v1_y_te_rescaled_f = os.path.join(idp_cv_dir,'v1_y_rescaled_'+str(sex)+'.txt')
    v2_y_te_rescaled_f = os.path.join(idp_cv_dir,'v2_y_rescaled_'+str(sex)+'.txt')
    np.savetxt(v1_y_te_rescaled_f,v1_y_rescaled)
    np.savetxt(v2_y_te_rescaled_f,v2_y_rescaled)

    #s2_rescaled_f = os.path.join(idp_visit_dir,'s2_rescaled_'+str(sex)+'.txt')
    #np.savetxt(s2_rescaled_f,s2_rescaled)


    # V1 plot the (adjusted) data points
    v1_plot = v1_cont.iloc[v1_common_s_id]['age'].to_numpy()
    v2_plot = v2_cont.iloc[v2_common_s_id]['age'].to_numpy()

    x_coords = np.concatenate((v1_plot[np.newaxis,:],v2_plot[np.newaxis,:]),axis=0)
    y_coords = np.array([np.concatenate(v1_y_rescaled),np.concatenate(v2_y_rescaled)])

    plt.figure(figsize=(8,6))
    plt.plot(x_coords, y_coords, color='gray')

    plt.scatter(v1_plot, v1_y_rescaled, color='darkgreen', label='V1', alpha = 0.9)
    plt.scatter(v2_plot, v2_y_rescaled, color='saddlebrown', label = 'V2', alpha = 0.9)

    # plot the median of the dummy data
    plt.plot(xx, med, clr)
    
    # fill the gaps in between the centiles
    junk, pr_int25 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.25,0.75])
    junk, pr_int95 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.05,0.95])
    junk, pr_int99 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.01,0.99])
    plt.fill_between(xx, pr_int25[:,0], pr_int25[:,1], alpha = 0.1,color=clr)
    plt.fill_between(xx, pr_int95[:,0], pr_int95[:,1], alpha = 0.1,color=clr)
    plt.fill_between(xx, pr_int99[:,0], pr_int99[:,1], alpha = 0.1,color=clr)
            
    # make the width of each centile proportional to the epistemic uncertainty
    junk, pr_int25l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.25,0.75])
    junk, pr_int95l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.05,0.95])
    junk, pr_int99l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.01,0.99])
    junk, pr_int25u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.25,0.75])
    junk, pr_int95u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.05,0.95])
    junk, pr_int99u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.01,0.99])    
    plt.fill_between(xx, pr_int25l[:,0], pr_int25u[:,0], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int95l[:,0], pr_int95u[:,0], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int99l[:,0], pr_int99u[:,0], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int25l[:,1], pr_int25u[:,1], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int95l[:,1], pr_int95u[:,1], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int99l[:,1], pr_int99u[:,1], alpha = 0.3,color=clr)

    # plot actual centile lines
    plt.plot(xx, pr_int25[:,0],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int25[:,1],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int95[:,0],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int95[:,1],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int99[:,0],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int99[:,1],color=clr, linewidth=0.5)
    
    plt.xlabel('Age')
    plt.ylabel(idp) 
    plt.title(idp)
    plt.xlim((10,60))
    plt.legend()
    #plt.savefig(os.path.join('centiles_' + str(sex)),  bbox_inches='tight')
    #plt.show()
    plt.savefig(os.path.join(images_dir,idp+'_'+str(sex)+'.png'))
    plt.close()
    

    
    

Running IDP 0 lh_G&S_frontomargin_thickness :
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
Running IDP 1 lh_G&S_occipital_inf_thickness :
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
Running IDP 2 lh_G&S_paracentral_thickness :
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
Running IDP 3 lh_G&S_subcentral_thickness :
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
Running IDP 4 lh_G&S_transv_frontopol_thickness :
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1
Writing outputs ...
Running IDP 5 lh_G&S_cingul-Ant_thickness :
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by mo